# Ranking for Maximum Impact

We have a list of charities, $x_1,\ldots,x_n$, and a list of expert judges $e_1,\ldots,e_k$. Each expert judges some charities and gives them a numerical score $e_j(x_i)$. The scores are based on that expert's subjective judgement across a dozen or so criteria, each are scored with a number 1-5 and then are weightedly summed. 

Our question is: how do we rank the charities? 

## model

Let's assume that each charity $x_i$ has a "true" value of $y_i$, which the experts are trying to estimate. 

The experts are additively and multiplicatively biased, and their scores are noisy. We model this as follows:

$$ e_j(x_i) = m_j y_i + b_j + \varepsilon_{j,i}$$

where $m_j$ is the constant multiplicative bias for each expert $e_j$, $b_j$ is the constant additive bias, and $\varepsilon_{j,i} \sim \text{N}(0,\sigma_j)$ is the noise. All variables are assumed to be independent.

## optimization

We are looking for a solution that'd be the "best" fit. A solution would include all of the $y_i$'s, $m_j$'s, $b_j$'s, and $\sigma_j$'s. Note that we have a couple of degrees of freedom here, which amount to changing all $y_i$ by a single linear transformation. We could solve that by normalizing the $y_i$ to have mean 0 and variance 1, for example.

One option would be to define the best fit as the solution that maximizes the likelihood of the available estimates $e_j(x_i)$. After taking logarithm and dropping the constant terms, we get the following expression we want to maximize:

$$ \sum_{i,j} \left( -\frac{1}{2} \log(2\pi\sigma_j^2) - \frac{1}{2\sigma_j^2} (e_j(x_i) - m_j y_i - b_j)^2 \right) $$

where the sum is over all charity/expert pair that's available. Equivalently, we can minimize

$$ 2\sum_j \log(\sigma_j) + \sum_{i,j} \frac{1}{2\sigma_j^2} (e_j(x_i) - m_j y_i - b_j)^2 $$

where the multiplier 2 is the number of charities each expert has (in our case).

This is nearly a convex optimization problem, so we can use a standard solver to find the solution.


[$m_jy_i$ isn't convex. Also $1/\sigma_j^2$ isn't convex]

For simplicity, let's assume that all experts have the same error distribution, i.e. $\sigma_j = \sigma$ for all $j$. Thus, we get a least squares problem:

$$ \min_{y,m,b} \sum_{i,j} (e_j(x_i) - m_j y_i - b_j)^2 $$

In experiments below, this has problems converging. To solve this, we will estimate $m_j$ in two ways. First we will take it to be 1, and second we will estimate it from the data directly for each expert to normalize their own grades. We expect the truth to be somewhere in between, so we'll see how that affects the results.

We also add a regularization term to make sure $b_j$ is small and we normalize the $e_j(x_i)$'s to have mean 0 and variance 1.

Now, we can represent the problem as a matrix equation:

$$ \min_{y,b} \| E - A(y,b) \|_F^2 + \lambda \| b \|_2^2 $$

where $E$ is the matrix of expert scores, $A$ is a $nk,n+k$ matrix taking the concatenation of the vectors $y$ and $b$ to the estimate at each index, and $\lambda$ is the regularization parameter. The matrix $A$ is defined as follows:

$$ A_{(i,j),l} = \begin{cases} \delta_{i,l}m_j & \text{if } l \leq n \\ \delta_{j,l} & \text{otherwise} \end{cases} $$

where $\delta$ is the Kronecker delta (1 if the indices are equal, 0 otherwise). We can then add extra rows to represent the regularization.

In [76]:
import numpy as np

df = pd.read_csv('./df_grades - Sheet1.csv')
df['norm_grade'] = (df.grade - df.grade.mean()) / df.grade.std()
df['m'] = 1
charities = sorted(df['org'].unique())
experts = sorted(df['judge'].unique())
n = len(charities)
k = len(experts)
df.columns

Index(['org', 'judge', 'grade', 'norm_grade', 'm'], dtype='object')

In [77]:
def create_matrix():
    a = np.zeros([n*k + k, n+k])
    for row in df.iterrows():
        i = charities.index(row[1]['org'])
        j = experts.index(row[1]['judge'])
        m = row[1]['m']
        a[i*k + j, i] = m
        a[i*k + j, n + j] = 1
    for j in range(k):
        a[n*k + j, n + j] = 1
    return a       

def create_target_vector():
    e = np.zeros(n*k + k)
    
    for row in df.iterrows():
        i = charities.index(row[1]['org'])
        j = experts.index(row[1]['judge'])
        e[i*k + j] = row[1]['norm_grade']

    # the last k values are 0

    return e


In [78]:
a = create_matrix()
e = create_target_vector()
sol = np.linalg.lstsq(a, e, rcond=None)